## 0. Configuartion

In [1]:
# %load_ext autoreload
import os
import sys

device = 'cuda'
wait_for_v100 = False
ds_to_use = [
    # 'HP1-FvM',
    # 'HP1-RB',
    # 'HP2-FvM',
    # 'HP2-RB',
    # 'HP3-FvM',
    # 'HP3-RB',
    # 'HP4-FvM',
    'HP4-RB',
    # 'HP5-FvM',
    # 'HP5-RB',
    # 'HP6-FvM',
    # 'HP6-RB',
    # 'HP7-FvM',
    # 'HP7-RB'
]
home = os.getenv("HOME")
use_gdrive = False
trained_model_directory = 'trained_model'
# os.environ['WANDB_API_KEY'] = '6cec6a71175dec59b32a9067a0664dec76208f2a'
# os.environ['WANDB_NAME'] = 'GermanWaveToVec Training'
# os.environ['WANDB_ENTITY'] ='wandb.ai/elunrast'
# os.environ['WANDB_PROJECT'] ='GermanWaveToVec'

## ----- Training arguments  ----- ##
per_device_train_batch_size = 4
# TODO: aus freiem Grafikkarten Speicher ableiten (Graphikkarte mit 11GB)
per_device_eval_batch_size  = per_device_train_batch_size // 2
gradient_accumulation_steps = 2
logging_steps               = 100
max_steps                   = per_device_train_batch_size * logging_steps
max_trainingset_size        = max_steps * per_device_train_batch_size # TODO freiem Grafikkarten Speicher ableiten
max_steps                   = max_trainingset_size // per_device_train_batch_size # Schritte pro Epoche
warmup_steps                = max_steps // 4
save_steps                  = max_steps // 2
eval_steps                  = save_steps
max_sample_size             = 1200   # None -> no restriction
max_trainingset_size        = per_device_train_batch_size * max_steps   # None -> no restriction
fixed_training_set_size     = 100 * per_device_eval_batch_size          # None -> 20% der Originaldatensatz Größe 
num_train_epochs            = 3  # Anzahl Durchläufe für ein Dataset
max_rounds                  = 4  # Anzahl Trainingsrunden über alle used_datasets

learning_rate = per_device_train_batch_size * ((3e-4)/16)  # 3e-4/4 

## 1. Setup Directories

In [2]:
local_nlp_base_dir = f'{home}/shared'
local_data_base_dir = f'{local_nlp_base_dir}/NLP-Data'
local_data_dir = f'{local_data_base_dir}/audio'
extern_nlp_base_dir = None
extern_data_base_dir = None
extern_data_dir = None

runs_on_colab = (home == '/root')
print( f'runs on colab: {runs_on_colab}')

if not runs_on_colab:
    os.environ['http_proxy'] = 'http://192.168.8.50:3128'
    os.environ['https_proxy'] = 'http://192.168.8.50:3128'
else:
    # to get access to the datasets we use gdrive
    use_gdrive = True
    # install packages
    !pip install datasets==1.4.1
    !pip install transformers==4.4.0
    !pip install jiwer
    !pip install torchaudio
    !pip install librosa
    # create local directories
    !mkdir $local_nlp_base_dir
    !mkdir $local_data_base_dir

if use_gdrive:
    gdrive_base = '/content/gdrive'    
    extern_nlp_base_dir = f'{gdrive_base}/MyDrive'
    extern_data_base_dir = f'{extern_nlp_base_dir}/NLP-Data'
    extern_data_dir = f'{extern_data_base_dir}/audio'

    if not os.path.isdir(gdrive_base):
        from google.colab import drive
        drive.mount(gdrive_base)

if not os.path.isdir(local_data_dir):
    !mkdir $local_data_dir

if extern_nlp_base_dir:
    model_dir = f'{extern_nlp_base_dir}/NLP-Models/GermanWave2Vec'
else:
    model_dir = f'{local_nlp_base_dir}/NLP-Models/GermanWave2Vec'

# Use the Model from this Directory (Base: .../NLP-Models/GermanWave2Vec/)
# None -> Start from 'facebook/wav2vec2-large-xlsr-53-german'
trained_model_path = f'{model_dir}/{trained_model_directory}'

git_views_dir = f'{local_nlp_base_dir}/gitviews/extern'
git_view_path = f'{git_views_dir}/GermanWave2Vec'

if not os.path.isdir(git_views_dir):
    !mkdir $git_views_dir
    !cd $git_views_dir; git clone https://github.com/ElUnrast/GermanWave2Vec.git

runs on colab: False


In [3]:
if runs_on_colab:
    !cd $git_view_path; git fetch --all; git reset --hard origin/main

script_path = f'{git_views_dir}/GermanWave2Vec/python'
print(f'Script-Path: {script_path}')
sys.path.insert(0, script_path)

Script-Path: /home/ki-mo/shared/gitviews/extern/GermanWave2Vec/python


## 2. Check Runtime Properties

In [4]:
if 'cuda' == device:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)

    if runs_on_colab:
        if wait_for_v100 and not gpu_info.find('V100') >= 0:
            print('The current GPU is not a V100')
            print('Since you want to wait for a V100 the current session is aborted')
            raise ValueError

        if gpu_info.find('failed') >= 0:
            print('For training, please use a VM with GPU!')
            raise ValueError
            
        from psutil import virtual_memory
        ram_gb = virtual_memory().total / 1e9
        print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

        if ram_gb < 20:
            print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
            print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
            print('re-execute this cell.')
        else:
            print('You are using a high-RAM runtime!')
            
    print(gpu_info)

Mon May 17 13:52:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:09:00.0 Off |                  N/A |
| 37%   40C    P8    13W / 260W |     91MiB / 11016MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 3 Install packages and do Imports

In [5]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import torch
import torchaudio
from torch import nn
from torch.cuda.amp import autocast

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
from transformers import Trainer, TrainingArguments
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.trainer_pt_utils import LengthGroupedSampler, DistributedLengthGroupedSampler

import json
import collections
import librosa
import numpy as np
import pandas as pd
import sklearn
import jiwer
from jiwer import wer
from datasets import load_metric
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split

In [6]:
# %aimport SnippetDatasets
# %aimport GermanSpeechDatasetWidgetFactory
# %aimport GermanSpeechToTextTranslater
# %autoreload 2
from SnippetDatasets import SnippetDatasets
from GermanSpeechDatasetWidgetFactory import GermanSpeechDatasetWidgetFactory
from GermanSpeechToTextTranslater import GermanSpeechToTextTranslater

## 4 Initialize Helper Classes

In [7]:
my_datasets = SnippetDatasets(
    runs_on_colab, 
    local_audio_base_dir=local_data_dir, 
    git_repository=git_view_path,
    extern_audio_base_dir=extern_data_dir
)

In [8]:
model_path = None
checkpoint = None

if checkpoint:
    model_path = f'{trained_model_path}/{checkpoint}'
elif os.path.isfile(f'{trained_model_path}/pytorch_model.bin'):
    model_path = trained_model_path

print(f'Initialize Model from Path: {model_path}')
translator = GermanSpeechToTextTranslater(model_name=model_path, ds_handler=my_datasets)

Initialize Model from Path: /home/ki-mo/shared/NLP-Models/GermanWave2Vec/trained_model
Using Model: /home/ki-mo/shared/NLP-Models/GermanWave2Vec/trained_model
Loading processor
Loading metric
json loaded: {'trained_epochs': 4}
Saved Epoch: 4
Loading model. Epoche 4


## 4. Choose Dataset

In [9]:
widget_factory = GermanSpeechDatasetWidgetFactory(my_datasets, ds_to_use)
print('Please choose Datasets to use')
ds_checkboxes_widget = widget_factory.create_dataset_choice_widget()
display(ds_checkboxes_widget)

Please choose Datasets to use


Box(children=(Checkbox(value=False, description='HP1-FvM', indent=False), Checkbox(value=False, description='H…

In [10]:
translator.train(
        trained_model_path=trained_model_path,
        dataset_loader=my_datasets, 
        ds_to_train=widget_factory.get_used_datasets(), 
        max_trainingset_size=max_trainingset_size, 
        max_rounds=max_rounds, 
        num_train_epochs=num_train_epochs,
        num_steps_per_epoche=max_steps,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        logging_steps=logging_steps,
        learning_rate=learning_rate,
        warmup_steps=warmup_steps,
        early_stopping_value=0.15
)

Starting round 0 of 4
-----------------------------
Loading Dataset: HP4-RB - content-translated-with_original.csv
Pruning Dataset HP4-RB with 15499 Entries
 - 15499 Entries left after Length Cut (min=31, max=4000)
 - 15499 Entries left after Action Cut
Dataset - HP4-RB loaded with 15499 Entries
**************************************
Starting round 0 of 4, epoche 0 of 3
Splitting Dataset HP4-RB with 15499 Entries
aktual trained epoches: 4
old trained epoches: 4
Actual number of bad translated 5287
Actual WER: 5.537159475856961
 - 2500 left after Entries Max Samples Cut (max=2500)
Creating Trainer for HP4-RB


W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.


Training of Dataset: HP4-RB


Step,Training Loss,Validation Loss


Save Model
**************************************
Starting round 0 of 4, epoche 1 of 3
Splitting Dataset HP4-RB with 15499 Entries
aktual trained epoches: 5
old trained epoches: 4
Translate all


...

WER: 0.14232574858126287
No. of bad translated snippets: 9949
Saving diff files
Actual number of bad translated 9949
Actual WER: 0.14232574858126287
 - 2500 left after Entries Max Samples Cut (max=2500)
Creating Trainer for HP4-RB
Training of Dataset: HP4-RB


Step,Training Loss,Validation Loss


Save Model
**************************************
Starting round 0 of 4, epoche 2 of 3
Splitting Dataset HP4-RB with 15499 Entries
aktual trained epoches: 6
old trained epoches: 5
Translate all


/home/ki-mo/python_env/qca38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


RuntimeError: CUDA out of memory. Tried to allocate 1.56 GiB (GPU 0; 10.76 GiB total capacity; 6.40 GiB already allocated; 1.36 GiB free; 7.83 GiB reserved in total by PyTorch)

In [ ]:
print('finished')
widget_factory.play_audio_file(audio_url="http://www.w3schools.com/html/horse.ogg")